In [1]:
import pickle
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA

## 6.1 Data Exploration: 

- I strongly encourage you to first explore the data a bit. At the very least, answer the following questions:

In [2]:
X,y = pickle.load(open('cosmic.p','rb'))

In [3]:
X.shape, y.shape

((6755, 18703), (6755,))

### (a) How many of the 6,755 samples belong to each of the six types of cancer?

In [4]:
samples_per_cancer = {i: (y == i).sum() for i in range(1, 7)}
samples_per_cancer

{1: 608, 2: 1867, 3: 1399, 4: 1363, 5: 1006, 6: 512}

### (b) Count the total number of mutations for each **sample** (i.e. each row of X). What’s the smallest number? The largest number? The average?

In [5]:
# number of mutations in each samples 
mutations_per_sample = np.sum(X, axis=1)
min_mutations_sample = np.min(mutations_per_sample)
max_mutations_sample = np.max(mutations_per_sample)
average_mutations_sample = np.mean(mutations_per_sample)

print(f"Mutation per sample: {mutations_per_sample}")
print(f"Minimum mutation per sample: {min_mutations_sample}")
print(f"Maximum mutation per sample: {max_mutations_sample}")
print(f"Average mutation per sample: {average_mutations_sample}")

Mutation per sample: [ 189  950   80 ... 1560   87   66]
Minimum mutation per sample: 45
Maximum mutation per sample: 5938
Average mutation per sample: 241.75292376017765


This shows the number of mutations in DNA of each cancer patients.

### (c) Count the total number of mutations for each **gene** (i.e. each column of X). What’s the smallest number? The largest number? The average?

In [6]:
# number of mutations in each gene
mutations_per_gene = np.sum(X, axis=0)
min_mutations_gene = np.min(mutations_per_gene)
max_mutations_gene = np.max(mutations_per_gene)
average_mutations_gene = np.mean(mutations_per_gene)

print(f"Mutation per gene: {mutations_per_gene}")
print(f"Minimum mutation per gene: {min_mutations_gene}")
print(f"Maximum mutation per gene: {max_mutations_gene}")
print(f"Average mutation per gene: {average_mutations_gene}")

Mutation per gene: [ 84  78  56 ... 120 102  35]
Minimum mutation per gene: 30
Maximum mutation per gene: 1768
Average mutation per gene: 87.31438806608566


This shows the number of mutations in each genes.

## 6.2 Baseline (No PCA): 

- Let’s get a sense of how well we can predict the type of cancer from mutations. Treat the first 4,500 rows of the data as your training set (and the remaining 2,255 rows as your test set). Now there are 6 types of cancer here: for each type of cancer, you will build a separate binary classifier which aims to distinguish that type of cancer from the other five. 
- For each of these 6 separate binary classification tasks, at the very least, try each of the following algorithms, and report the out-of-sample AUC:

In [7]:
# manually splitting the data

X_train = X[:4500]
X_test = X[4500:]

y_train = y[:4500]
y_test = y[4500:]

### (a) Logistic Regression

In [8]:
# making dictionary to list all type of cancers and corresponding AUC

auc_scores_lr = {}

for cancer_type in range(1, 7):
    y_train_binary = (y_train == cancer_type).astype(int)
    y_test_binary = (y_test == cancer_type).astype(int)

    model_lr = LogisticRegression(solver='liblinear', random_state=0)
    model_lr.fit(X_train, y_train_binary)

    y_pred_prob = model_lr.predict_proba(X_test)[:, 1]
    auc = roc_auc_score(y_test_binary, y_pred_prob)
    auc_scores_lr[cancer_type] = auc

auc_scores_lr

{1: 0.9596146404466713,
 2: 0.9549911317873117,
 3: 0.9248159793489283,
 4: 0.9406775286257196,
 5: 0.8847479752525764,
 6: 0.9151598522076091}

In [9]:
average_auc_lr = sum(auc_scores_lr.values()) / len(auc_scores_lr)
print("Average AUC for Logistic Regression:", average_auc_lr)

Average AUC for Logistic Regression: 0.9300011846114695


my Logistic Regression model (binary classifier) distinguishes the type 1 (Kidney Cancer) the best.

### (b) Logistic Regression with L1 penalty

In [10]:
auc_scores_lr_l1 = {}

for cancer_type in range(1, 7):
    y_train_binary = (y_train == cancer_type).astype(int)
    y_test_binary = (y_test == cancer_type).astype(int)

    model_lr_l1 = LogisticRegression(solver='liblinear', penalty='l1', random_state=0)
    model_lr_l1.fit(X_train, y_train_binary)

    y_pred_prob = model_lr_l1.predict_proba(X_test)[:, 1]
    auc = roc_auc_score(y_test_binary, y_pred_prob)
    auc_scores_lr_l1[cancer_type] = auc

auc_scores_lr_l1

{1: 0.9484943437379073,
 2: 0.9437528477033579,
 3: 0.8946157280036489,
 4: 0.9067789735547023,
 5: 0.8858894333158667,
 6: 0.9081200998711852}

In [11]:
average_auc_lr_l1 = sum(auc_scores_lr_l1.values()) / len(auc_scores_lr_l1)
print("Average AUC for Logistic Regression:", average_auc_lr_l1)

Average AUC for Logistic Regression: 0.9146085710311115


my Logistic Regression model with L1 penalty (Lasso) also distinguishes the type 1 (Kidney Cancer) the best.

### (c) Random Forest

In [12]:
auc_scores_rf = {}

for cancer_type in range(1, 7):
    y_train_binary = (y_train == cancer_type).astype(int)
    y_test_binary = (y_test == cancer_type).astype(int)
    
    model_rf = RandomForestClassifier(n_estimators=10, random_state=0, n_jobs=-1)
    model_rf.fit(X_train, y_train_binary)

    y_pred_prob = model_rf.predict_proba(X_test)[:, 1]
    auc = roc_auc_score(y_test_binary, y_pred_prob)
    auc_scores_rf[cancer_type] = auc

auc_scores_rf

{1: 0.7763565229190433,
 2: 0.8757516470236244,
 3: 0.7724858193710545,
 4: 0.7864747469284812,
 5: 0.7052801245338408,
 6: 0.7873947085235073}

In [13]:
average_auc_rf = sum(auc_scores_rf.values()) / len(auc_scores_rf)
print("Average AUC for Logistic Regression:", average_auc_rf)

Average AUC for Logistic Regression: 0.7839572615499252


my Random Forest model distinguishes the type 2 (Large Intestine) the best.

 It is concluded that the Normal Logistic Regression model shows the best performance for classifying most types of cancer, as it achieves the highest average AUC score.

## 6.3 PCA: 

- Repeat part 6.2, this time pre-processing the data using PCA. You will need to tune the number of components. I recommend you do this with the simplest possible method for validation: manually split your training set into a “training” and “validation” set (say 3,000 rows for the training set, and 1,500 rows for the validation set). Once again, at the very least, try each of the following algorithms, and report the out-of-sample AUC (so 18 AUCs in total):

#### But Why PCA before training the model? and What matters with the different components in PCA?

PCA is a powerful tool for pre-processing and feature engineering in machine learning. It helps to make the data more manageable for models, potentially improving performance, reducing overfitting, and saving computational resources. Testing different numbers of components is crucial to leverage the benefits of PCA fully, as it helps in identifying the sweet spot where the reduced feature set captures the essence of the original data without including too much noise or losing significant information.

### (a) Logistic Regression

In [14]:
# Manually splitted the original training set in to training and validation

X_new_train = X_train[:3000, :]
X_validation = X_train[3000:4500, :]
y_new_train = y_train[:3000]
y_validation = y_train[3000:4500]

pca_components_list = [20, 50, 100, 1000, 3000]

auc_scores = {n_components: {} for n_components in pca_components_list}

for n_components in pca_components_list:
    
    pca = PCA(n_components=n_components)
    X_new_train_pca = pca.fit_transform(X_new_train)
    X_validation_pca = pca.transform(X_validation)
    
    for cancer_type in range(1, 7):
        y_new_train_binary = (y_new_train == cancer_type).astype(int)
        y_validation_binary = (y_validation == cancer_type).astype(int)
        
        model = LogisticRegression(solver='liblinear', random_state=0)
        model.fit(X_new_train_pca, y_new_train_binary)
        
        y_pred_prob = model.predict_proba(X_validation_pca)[:, 1]
        auc = roc_auc_score(y_validation_binary, y_pred_prob)
        
        auc_scores[n_components][cancer_type] = auc

for n_components, scores in auc_scores.items():
    average_auc = sum(scores.values()) / len(scores)
    print(f"PCA Components: {n_components}, AUC Scores: {scores}")
    print(f"Average AUC: {average_auc}\n")

PCA Components: 20, AUC Scores: {1: 0.8168370217554388, 2: 0.7257229311695679, 3: 0.6800764781126023, 4: 0.8130503755503755, 5: 0.6234462391475035, 6: 0.7788}
Average AUC: 0.7396555076225813

PCA Components: 50, AUC Scores: {1: 0.8432166635408852, 2: 0.8058577462378761, 3: 0.713915609052975, 4: 0.8889806181472848, 5: 0.6726819196681001, 6: 0.8006357142857142}
Average AUC: 0.7875480451554725

PCA Components: 100, AUC Scores: {1: 0.8667010502625657, 2: 0.8432313289685792, 3: 0.7464458500888368, 4: 0.8916542562375898, 5: 0.6964919819962455, 6: 0.8371857142857143}
Average AUC: 0.8136183636399218

PCA Components: 1000, AUC Scores: {1: 0.9284684452363091, 2: 0.9448964796786773, 3: 0.8747035391409463, 4: 0.9133644133644134, 5: 0.817681289601313, 6: 0.9242142857142857}
Average AUC: 0.9005547421226575

PCA Components: 3000, AUC Scores: {1: 0.9418663259564891, 2: 0.9546834052286993, 3: 0.8907509620611231, 4: 0.9160407493740828, 5: 0.8553528212452058, 6: 0.9106857142857143}
Average AUC: 0.9115633

### (b) Logistic Regression with L1 penalty

In [15]:
pca_components_list = [20, 50, 100, 1000, 3000]

auc_scores_lr_l1 = {n_components: {} for n_components in pca_components_list}

for n_components in pca_components_list:
    pca_l1 = PCA(n_components=n_components)
    X_new_train_pca_l1 = pca_l1.fit_transform(X_new_train)
    X_validation_pca_l1 = pca_l1.transform(X_validation)
    
    for cancer_type in range(1, 7):
        y_new_train_binary_l1 = (y_new_train == cancer_type).astype(int)
        y_validation_binary_l1 = (y_validation == cancer_type).astype(int)
        
        model_lr_l1 = LogisticRegression(solver='liblinear', penalty='l1', random_state=0)
        model_lr_l1.fit(X_new_train_pca_l1, y_new_train_binary_l1)
        
        y_pred_prob_l1 = model_lr_l1.predict_proba(X_validation_pca_l1)[:, 1]
        auc_l1 = roc_auc_score(y_validation_binary_l1, y_pred_prob_l1)
        
        auc_scores_lr_l1[n_components][cancer_type] = auc_l1

for n_components, scores in auc_scores_lr_l1.items():
    average_auc = sum(scores.values()) / len(scores)
    print(f"PCA Components: {n_components}, AUC Scores: {scores}")
    print(f"Average AUC: {average_auc}\n")

PCA Components: 20, AUC Scores: {1: 0.8147388409602401, 2: 0.7297619798092596, 3: 0.6806795410231518, 4: 0.81285882327549, 5: 0.6236401058512203, 6: 0.7771857142857143}
Average AUC: 0.7398108342008459

PCA Components: 50, AUC Scores: {1: 0.8401162790697675, 2: 0.8001031150656064, 3: 0.7086124414853197, 4: 0.8902945048778382, 5: 0.6730728841872622, 6: 0.7917500000000001}
Average AUC: 0.7839915374476324

PCA Components: 100, AUC Scores: {1: 0.8705985090022507, 2: 0.8384538143568592, 3: 0.7511189115660977, 4: 0.8932433307433308, 5: 0.6992028847365515, 6: 0.8371857142857142}
Average AUC: 0.814967194115134

PCA Components: 1000, AUC Scores: {1: 0.9174207614403601, 2: 0.9449347538296666, 3: 0.8802690255531007, 4: 0.9074640637140636, 5: 0.8197815122249112, 6: 0.9183214285714285}
Average AUC: 0.8980319242222552

PCA Components: 3000, AUC Scores: {1: 0.9206911102775693, 2: 0.9543141722426852, 3: 0.8761828010784819, 4: 0.9152070922904256, 5: 0.8477273975656805, 6: 0.9221714285714286}
Average AUC

### (c) Random Forest

In [16]:
pca_components_list = [20, 50, 100, 1000, 3000]

auc_scores_rf_pca = {n_components: {} for n_components in pca_components_list}

for n_components in pca_components_list:
    pca_rf = PCA(n_components=n_components)
    X_new_train_pca_rf = pca_rf.fit_transform(X_new_train)
    X_validation_pca_rf = pca_rf.transform(X_validation)
    
    for cancer_type in range(1, 7):
        y_new_train_binary_rf = (y_new_train == cancer_type).astype(int)
        y_validation_binary_rf = (y_validation == cancer_type).astype(int)
        
        model_rf_pca = RandomForestClassifier(n_estimators=10, random_state=0, n_jobs=-1)
        model_rf_pca.fit(X_new_train_pca_rf, y_new_train_binary_rf)
        
        y_pred_prob_rf = model_rf_pca.predict_proba(X_validation_pca_rf)[:, 1]
        auc_rf = roc_auc_score(y_validation_binary_rf, y_pred_prob_rf)
        
        auc_scores_rf_pca[n_components][cancer_type] = auc_rf

for n_components, scores in auc_scores_rf_pca.items():
    average_auc = sum(scores.values()) / len(scores)
    print(f"PCA Components: {n_components}, AUC Scores: {scores}")
    print(f"Average AUC: {average_auc}\n")

PCA Components: 20, AUC Scores: {1: 0.7245815360090022, 2: 0.7563523833538963, 3: 0.6627972383504742, 4: 0.8465423465423465, 5: 0.6448055032294961, 6: 0.6693357142857143}
Average AUC: 0.7174024536284883

PCA Components: 50, AUC Scores: {1: 0.7304482370592649, 2: 0.7700230095190066, 3: 0.6562771276897823, 4: 0.8570628831045497, 5: 0.6609917574339803, 6: 0.7443535714285714}
Average AUC: 0.7365260977058593

PCA Components: 100, AUC Scores: {1: 0.718978572768192, 2: 0.7773738979295937, 3: 0.6677921136678935, 4: 0.8647492553742554, 5: 0.6512822666894998, 6: 0.7033071428571429}
Average AUC: 0.7305805415477629

PCA Components: 1000, AUC Scores: {1: 0.7852031367216803, 2: 0.7789251267549824, 3: 0.6837908588643486, 4: 0.8213559203142538, 5: 0.6558413653385721, 6: 0.830867857142857}
Average AUC: 0.7593307108561157

PCA Components: 3000, AUC Scores: {1: 0.4375175825206301, 2: 0.6737691483325978, 3: 0.5210152550577508, 4: 0.7990373823707158, 5: 0.4984393730350801, 6: 0.43273571428571433}
Average A

## 6.4 Summary: 

- What is your final proposed procedure for achieving the highest out-of-sample AUC averaged across all six types of cancer? What is your proposed procedure’s average out-of-sample AUC?

After calculating the average AUC values for each algorithm (Logistic Regression, Logistic Regression with L1 regularization, and Random Forest) with and without PCA, I concluded that the baseline Logistic Regression model performs the best in classifying the Cancer Type based on the gene mutation status. The AUC values for each type of cancer were {1: 0.9596146404466713, 2: 0.9549911317873117, 3: 0.9248159793489283, 4: 0.9406775286257196, 5: 0.8847479752525764, 6: 0.9151598522076091}, and the average AUC was 0.9300011846114695. This means that the model is able to discriminate between positive and negative cases of cancer with about 93% accuracy for each type of cancer